Single-layer fPEPS amplitude TN boundary contraction test (requires my vmc_torch code)

In [16]:
# torch
import torch
from vmc_torch.hamiltonian_torch import spinful_Fermi_Hubbard_square_lattice_torch
from vmc_torch.fermion_utils import generate_random_fpeps
from vmc_torch.fermion_utils import fPEPS

# Hamiltonian parameters
Lx = int(6)
Ly = int(6)
symmetry = 'Z2'
t = 1.0
U = 8.0
N_f = int(Lx*Ly)
n_fermions_per_spin = (N_f//2, N_f//2)
H = spinful_Fermi_Hubbard_square_lattice_torch(Lx, Ly, t, U, N_f, pbc=False, n_fermions_per_spin=n_fermions_per_spin)
# TN parameters
D = 4
chi = 16
dtype=torch.float64
# Load PEPS
peps = generate_random_fpeps(Lx, Ly, D=D, seed=2, symmetry=symmetry, Nf=N_f, spinless=False)[0]
# peps = sr.PEPS_fermionic_rand(symmetry, Lx, Ly, bond_dim=D, phys_dim=4)
peps = fPEPS(peps)
peps_np = peps.copy()
peps.apply_to_arrays(lambda x: torch.tensor(x, dtype=dtype))
for ts in peps.tensors:
    ts.data.phase_sync(inplace=True)


In [17]:
random_x = torch.tensor(H.hilbert.random_state())
print("random_x:", random_x)

random_x: tensor([0, 1, 2, 0, 1, 3, 1, 2, 3, 3, 2, 3, 2, 3, 2, 1, 1, 3, 0, 3, 1, 2, 0, 0,
        1, 0, 3, 2, 3, 1, 0, 2, 0, 0, 0, 3])


In [18]:
amp0 = peps.get_amp(random_x)
benchmark_amp = amp0.contract()
print(f'Benchmark amplitude: {benchmark_amp}')

Benchmark amplitude: -3558302558151.4043


In [19]:
row_no = 2
max_bond = 64
amp_fit = amp0.contract_boundary_from_ymin(yrange=(0, row_no), max_bond=max_bond, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=10)
# print(amp_fit.contract())
print(f'boundary fitting from ymin to col {row_no}: amp fit: {amp_fit.contract()}, benchmark amp: {benchmark_amp}')
amp_fit = amp0.contract_boundary_from_ymin(yrange=(0, row_no), max_bond=max_bond, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=13)
# print(amp_fit.contract())
print(f'boundary fitting from ymin to col {row_no}: amp fit: {amp_fit.contract()}, benchmark amp: {benchmark_amp}')

boundary fitting from ymin to col 2: amp fit: -3558302558151.439, benchmark amp: -3558302558151.4043
boundary fitting from ymin to col 2: amp fit: -3558302558151.3994, benchmark amp: -3558302558151.4043
boundary fitting from ymin to col 2: amp fit: -3558302558151.3994, benchmark amp: -3558302558151.4043


In [20]:
amp_fit = amp0.contract_boundary_from_ymax(yrange=(row_no, Ly-1), max_bond=max_bond, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=10)
# print(amp_fit.contract())
print(f'boundary fitting from ymax to col {row_no}: amp fit: {amp_fit.contract()}, benchmark amp: {benchmark_amp}')
amp_fit = amp0.contract_boundary_from_ymax(yrange=(row_no, Ly-1), max_bond=max_bond, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=13)
# print(amp_fit.contract())
print(f'boundary fitting from ymax to col {row_no}: amp fit: {amp_fit.contract()}, benchmark amp: {benchmark_amp}')

boundary fitting from ymax to col 2: amp fit: -3558302558151.3896, benchmark amp: -3558302558151.4043
boundary fitting from ymax to col 2: amp fit: -3558302558151.404, benchmark amp: -3558302558151.4043
boundary fitting from ymax to col 2: amp fit: -3558302558151.404, benchmark amp: -3558302558151.4043


In [21]:
amp_fit = amp0.contract_boundary_from_xmin(xrange=(0, row_no), max_bond=max_bond, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=10)
# print(amp_fit.contract())
print(f'boundary fitting from xmin to row {row_no}: amp fit: {amp_fit.contract()}, benchmark amp: {benchmark_amp}')
amp_fit = amp0.contract_boundary_from_xmin(xrange=(0, row_no), max_bond=max_bond, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=13)
# print(amp_fit.contract())
print(f'boundary fitting from xmin to row {row_no}: amp fit: {amp_fit.contract()}, benchmark amp: {benchmark_amp}')

boundary fitting from xmin to row 2: amp fit: -3558302558151.395, benchmark amp: -3558302558151.4043
boundary fitting from xmin to row 2: amp fit: -3558302558151.411, benchmark amp: -3558302558151.4043
boundary fitting from xmin to row 2: amp fit: -3558302558151.411, benchmark amp: -3558302558151.4043


In [22]:
amp_fit = amp0.contract_boundary_from_xmax(xrange=(row_no, Lx-1), max_bond=max_bond, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=10)
print(f'boundary fitting from xmax to row {row_no}: amp fit: {amp_fit.contract()}, benchmark amp: {benchmark_amp}')
amp_fit = amp0.contract_boundary_from_xmax(xrange=(row_no, Lx-1), max_bond=max_bond, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=13)
print(f'boundary fitting from xmax to row {row_no}: amp fit: {amp_fit.contract()}, benchmark amp: {benchmark_amp}')
tn_boundary_quimb_fit = amp_fit.select(['X3'])

boundary fitting from xmax to row 2: amp fit: -3558302558151.412, benchmark amp: -3558302558151.4043
boundary fitting from xmax to row 2: amp fit: -3558302558151.4165, benchmark amp: -3558302558151.4043
boundary fitting from xmax to row 2: amp fit: -3558302558151.4165, benchmark amp: -3558302558151.4043


Double-layer fPEPS norm TN boundary contraction test

In [24]:
import symmray as sr
import numpy as np

# fPEPS parameters
Lx = int(4)
Ly = int(4)
symmetry = 'Z2'
t = 1.0
U = 8.0
N_f = int(Lx*Ly)
D = 4
# contraction bond dimension
chi = 64
# Load PEPS
fpeps = sr.PEPS_fermionic_rand(symmetry, Lx, Ly, bond_dim=D, phys_dim=4)
fpeps.equalize_norms_()
for ts in fpeps.tensors:
    ts.data.phase_sync(inplace=True)
fpeps_norm = fpeps.make_norm()
benchmark_norm = fpeps_norm.contract_boundary_from_xmax(xrange=(0, Lx-1), max_bond=256, cutoff=0.0, mode='direct').contract()

In [26]:
print(f'Benchmark norm: {benchmark_norm}')
print('Boundary fMPS-MPO fitting contraction test:')
chi = 128
print('xmin:')
c_xmin_0 = fpeps_norm.contract_boundary_from_xmin(xrange=(0, 1), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=4).contract()
c_xmin_1 = fpeps_norm.contract_boundary_from_xmin(xrange=(0, 2), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=4).contract()
c_xmin_2 = fpeps_norm.contract_boundary_from_xmin(xrange=(0, 1), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=5).contract()
c_xmin_3 = fpeps_norm.contract_boundary_from_xmin(xrange=(0, 2), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=5).contract()
print(c_xmin_0, np.allclose(c_xmin_0, benchmark_norm, rtol=1e-4))
print(c_xmin_1, np.allclose(c_xmin_1, benchmark_norm, rtol=1e-4))
print(c_xmin_2, np.allclose(c_xmin_2, benchmark_norm, rtol=1e-4))
print(c_xmin_3, np.allclose(c_xmin_3, benchmark_norm, rtol=1e-4))
print('xmax:')
c_xmax_0 = fpeps_norm.contract_boundary_from_xmax(xrange=(2, 3), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=4).contract()
c_xmax_1 = fpeps_norm.contract_boundary_from_xmax(xrange=(1, 3), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=4).contract()
c_xmax_2 = fpeps_norm.contract_boundary_from_xmax(xrange=(2, 3), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=5).contract()
c_xmax_3 = fpeps_norm.contract_boundary_from_xmax(xrange=(1, 3), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=5).contract()
print(c_xmax_0, np.allclose(c_xmax_0, benchmark_norm, rtol=1e-4))
print(c_xmax_1, np.allclose(c_xmax_1, benchmark_norm, rtol=1e-4))
print(c_xmax_2, np.allclose(c_xmax_2, benchmark_norm, rtol=1e-4))
print(c_xmax_3, np.allclose(c_xmax_3, benchmark_norm, rtol=1e-4))
print('ymin:')
c_ymin_0 = fpeps_norm.contract_boundary_from_ymin(yrange=(0, 1), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=4).contract()
c_ymin_1 = fpeps_norm.contract_boundary_from_ymin(yrange=(0, 2), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=4).contract()
c_ymin_2 = fpeps_norm.contract_boundary_from_ymin(yrange=(0, 1), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=5).contract()
c_ymin_3 = fpeps_norm.contract_boundary_from_ymin(yrange=(0, 2), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=5).contract()
print(c_ymin_0, np.allclose(c_ymin_0, benchmark_norm, rtol=1e-4))
print(c_ymin_1, np.allclose(c_ymin_1, benchmark_norm, rtol=1e-4))
print(c_ymin_2, np.allclose(c_ymin_2, benchmark_norm, rtol=1e-4))
print(c_ymin_3, np.allclose(c_ymin_3, benchmark_norm, rtol=1e-4))
print('ymax:')
c_ymax_0 = fpeps_norm.contract_boundary_from_ymax(yrange=(2, 3), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=4).contract()
c_ymax_1 = fpeps_norm.contract_boundary_from_ymax(yrange=(1, 3), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=4).contract()
c_ymax_2 = fpeps_norm.contract_boundary_from_ymax(yrange=(2, 3), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=5).contract()
c_ymax_3 = fpeps_norm.contract_boundary_from_ymax(yrange=(1, 3), max_bond=chi, cutoff=0.0, mode='fit', tn_fit='zipup', bsz=2, max_iterations=5).contract()
print(c_ymax_0, np.allclose(c_ymax_0, benchmark_norm, rtol=1e-4))
print(c_ymax_1, np.allclose(c_ymax_1, benchmark_norm, rtol=1e-4))
print(c_ymax_2, np.allclose(c_ymax_2, benchmark_norm, rtol=1e-4))
print(c_ymax_3, np.allclose(c_ymax_3, benchmark_norm, rtol=1e-4))


Benchmark norm: 3.3371615675573256e+19
Boundary fMPS-MPO fitting contraction test:
xmin:
3.3373350559190094e+19 True
3.337310722517562e+19 True
3.3373350559190168e+19 True
3.3373107225175613e+19 True
xmax:
3.3373350559190094e+19 True
3.337310722517562e+19 True
3.3373350559190168e+19 True
3.3373107225175613e+19 True
xmax:
3.3370696778374013e+19 True
3.337004013664239e+19 True
3.3370696778374087e+19 True
3.3370040136642507e+19 True
ymin:
3.3370696778374013e+19 True
3.337004013664239e+19 True
3.3370696778374087e+19 True
3.3370040136642507e+19 True
ymin:
3.3371865421434814e+19 True
3.3372484741579207e+19 True
3.337186542143483e+19 True
3.3372484741579145e+19 True
ymax:
3.3371865421434814e+19 True
3.3372484741579207e+19 True
3.337186542143483e+19 True
3.3372484741579145e+19 True
ymax:
3.337177989562007e+19 True
3.3371097386071753e+19 True
3.337177989562015e+19 True
3.337109738607172e+19 True
3.337177989562007e+19 True
3.3371097386071753e+19 True
3.337177989562015e+19 True
3.337109738607172e